In [2]:
import re 

import pandas as pd
import plotly.express as px
from plotly import graph_objects as go

from watch import get_leaders

In [3]:
leaders_df = pd.DataFrame(get_leaders('/home/amos/.local/share/aspyr-media/Sid Meier\'s Civilization VI/Logs/GameCore.log'))
leaders_df = leaders_df[leaders_df['civilization_type'] == 'FULL_CIV']
leaders_df['civilization'] = leaders_df['civilization'].map(lambda x: f'CIVILIZATION_{x.strip()}')
leaders_df

,player_id,civilization,leader,civilization_type,player_type
0,0,CIVILIZATION_OTTOMAN,SULEIMAN,FULL_CIV,Human
1,1,CIVILIZATION_PHOENICIA,DIDO,FULL_CIV,AI
2,2,CIVILIZATION_CREE,POUNDMAKER,FULL_CIV,AI
3,3,CIVILIZATION_SUMERIA,GILGAMESH,FULL_CIV,AI
4,4,CIVILIZATION_CANADA,LAURIER,FULL_CIV,AI
5,5,CIVILIZATION_NORWAY,HARDRADA,FULL_CIV,AI
6,6,CIVILIZATION_JAPAN,HOJO,FULL_CIV,AI
7,7,CIVILIZATION_AZTEC,MONTEZUMA,FULL_CIV,AI
8,8,CIVILIZATION_FRANCE,CATHERINE DE MEDICI,FULL_CIV,AI
9,9,CIVILIZATION_KOREA,SEONDEOK,FULL_CIV,AI


In [4]:
def process_csv_pandas(input_filepath, num_expected_columns):
    """
    Reads a CSV file using pandas and puts extra entries into the final column.

    Args:
        input_filepath (str): Path to the input CSV file.
        output_filepath (str): Path to the output CSV file.
        num_expected_columns (int): The number of columns you expect (the last one will hold extras).
    """
    df = pd.read_csv(input_filepath, header=None, dtype=str)  # Read without header, as strings

    def combine_extras(row):
        if len(row) > num_expected_columns:
            extras = row[num_expected_columns - 1:].astype(str).str.cat(sep=' ')
            processed_row = list(row[:num_expected_columns - 1]) + [extras]
            return pd.Series(processed_row)
        else:
            return row

    processed_df = df.apply(combine_extras, axis=1)
    processed_df.columns = [f'col_{i+1}' for i in range(processed_df.shape[1])] # Optional: Rename columns
    return processed_df

In [27]:
ai_df = pd.read_csv('/home/amos/.local/share/aspyr-media/Sid Meier\'s Civilization VI/Logs/AI_CityBuild.csv', header=None, dtype=str, skiprows=1)
ai_df = ai_df.drop(columns=[12, 13], axis=1)  # Drop the first column
# df[' Player'] = df[' Player'].map(lambda x: x.strip())
ai_df.columns = ['Game Turn', 'Player', 'City', 'Food Adv.', 'Prod. Adv.', 'Construct', 'Food', 'Production', 'Gold', 'Science', 'Culture', 'Faith']
ai_df['Player'] = ai_df['Player'].map(lambda x: int(x.strip()))
ai_df

,Game Turn,Player,City,Food Adv.,Prod. Adv.,Construct,Food,Production,Gold,Science,Culture,Faith
0,1,0,LOC_CITY_NAME_ISTANBUL,-0.02,4.31,,YIELD_FOOD: 0.0,YIELD_PRODUCTION: 0.1,YIELD_GOLD: -0.1,YIELD_SCIENCE: 0.0,YIELD_CULTURE: 0.0,YIELD_FAITH: 0.0
1,1,2,LOC_CITY_NAME_MIKISIW_WACIHK,0.90,1.36,,YIELD_FOOD: 0.0,YIELD_PRODUCTION: 0.1,YIELD_GOLD: -0.1,YIELD_SCIENCE: 0.0,YIELD_CULTURE: 0.0,YIELD_FAITH: 0.0
2,1,2,LOC_CITY_NAME_MIKISIW_WACIHK,,,BUILDING_MONUMENT,Default Specialization,,,Value 20.9,NaN,NaN
3,1,3,LOC_CITY_NAME_URUK,0.72,0.38,,YIELD_FOOD: 0.1,YIELD_PRODUCTION: 0.0,YIELD_GOLD: 0.0,YIELD_SCIENCE: 0.0,YIELD_CULTURE: 0.0,YIELD_FAITH: -0.1
4,1,3,LOC_CITY_NAME_URUK,,,UNIT_SCOUT,Default Specialization,,,Value 34.8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
768,25,16,LOC_CITY_NAME_BANDAR_BRUNEI,,CONTRACT 760,SUBMITTED,UNIT_BUILDER,NO_BUILDING,NO_DISTRICT,VALUE 115.0,Build City Improvement,NaN
769,25,17,LOC_CITY_NAME_LISBON,,,NO BUILDS AT ALL (maybe city was nuked?),Generic Offense,NaN,NaN,NaN,NaN,NaN
770,25,17,LOC_CITY_NAME_LISBON,,,NO BUILDS AT ALL (maybe city was nuked?),Generic Offense,NaN,NaN,NaN,NaN,NaN
771,25,22,PURCHASE,,,UNIT_BUILDER,Build City Improvement,CONTRACT 734,NaN,NaN,NaN,NaN


In [6]:
player = ai_df[ai_df['Player'] == 2]
player

,Game Turn,Player,City,Food Adv.,Prod. Adv.,Construct,Food,Production,Gold,Science,Culture,Faith
1,1,2,LOC_CITY_NAME_MIKISIW_WACIHK,0.90,1.36,,YIELD_FOOD: 0.0,YIELD_PRODUCTION: 0.1,YIELD_GOLD: -0.1,YIELD_SCIENCE: 0.0,YIELD_CULTURE: 0.0,YIELD_FAITH: 0.0
2,1,2,LOC_CITY_NAME_MIKISIW_WACIHK,,,BUILDING_MONUMENT,Default Specialization,,,Value 20.9,NaN,NaN
49,2,2,LOC_CITY_NAME_MIKISIW_WACIHK,,CONTRACT 10,SUBMITTED,UNIT_CREE_OKIHTCITAW,NO_BUILDING,NO_DISTRICT,VALUE 18.0,Build Trigger Unit,NaN
50,2,2,LOC_CITY_NAME_MIKISIW_WACIHK,,CONTRACT 11,SUBMITTED,UNIT_SLINGER,NO_BUILDING,NO_DISTRICT,VALUE 26.0,Build Trigger Unit,NaN
69,3,2,LOC_CITY_NAME_MIKISIW_WACIHK,,,UNIT_SLINGER,Build Trigger Unit,CONTRACT 11,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
724,25,2,LOC_CITY_NAME_PIHTOKAHANAPIWIYIN,,CONTRACT 559,COMPLETED,UNIT_SETTLER,NO_BUILDING,NO_DISTRICT,VALUE 478.0,Settle City Op,NaN
725,25,2,LOC_CITY_NAME_PIHTOKAHANAPIWIYIN,,CONTRACT 652,IN PROGRESS,UNIT_BUILDER,NO_BUILDING,NO_DISTRICT,VALUE 102.0,Build City Improvement,NaN
726,25,2,LOC_CITY_NAME_PIHTOKAHANAPIWIYIN,,CONTRACT 711,IN PROGRESS,UNIT_BUILDER,NO_BUILDING,NO_DISTRICT,VALUE 274.0,Build City Improvement,NaN
727,25,2,LOC_CITY_NAME_PIHTOKAHANAPIWIYIN,,,UNIT_HORSEMAN,Default Specialization,,,Value 51.2,NaN,NaN


In [12]:
df = pd.read_csv('/home/amos/.local/share/aspyr-media/Sid Meier\'s Civilization VI/Logs/City_BuildQueue.csv')
df.columns = [x.strip() for x in df.columns]
df

,Game Turn,City,Production Added,Current Item,Current Production,Production Needed,Overflow
0,2,LOC_CITY_NAME_ISTANBUL,5.0,UNIT_SCOUT,5.0,15,0.0
1,2,LOC_CITY_NAME_MIKISIW_WACIHK,6.4,BUILDING_MONUMENT,6.4,30,0.0
2,2,LOC_CITY_NAME_URUK,9.6,UNIT_SCOUT,9.6,15,0.0
3,2,LOC_CITY_NAME_OTTAWA,8.0,UNIT_SCOUT,8.0,15,0.0
4,2,LOC_CITY_NAME_NIDAROS,9.6,UNIT_SCOUT,9.6,15,0.0
...,...,...,...,...,...,...,...
784,25,LOC_CITY_NAME_YEREVAN,2.8,BUILDING_GRANARY,11.4,32,0.0
785,25,LOC_CITY_NAME_ANTANANARIVO,3.5,UNIT_TRADER,27.5,25,2.5
786,26,LOC_CITY_NAME_ISTANBUL,9.9,UNIT_SCOUT,22.6,15,7.6
787,26,LOC_CITY_NAME_BURSA,7.2,UNIT_BUILDER,21.6,27,0.0


In [13]:
ai_df.columns

Index(['Game Turn', 'Player', 'City', 'Food Adv.', 'Prod. Adv.', 'Construct',
       'Food', 'Production', 'Gold', 'Science', 'Culture', 'Faith'],
      dtype='object')

In [14]:
df.columns

Index(['Game Turn', 'City', 'Production Added', 'Current Item',
       'Current Production', 'Production Needed', 'Overflow'],
      dtype='object')

In [30]:
leaders = ai_df[['Player', 'City']].drop_duplicates()
leaders

,Player,City
0,0,LOC_CITY_NAME_ISTANBUL
1,2,LOC_CITY_NAME_MIKISIW_WACIHK
3,3,LOC_CITY_NAME_URUK
5,4,LOC_CITY_NAME_OTTAWA
7,5,LOC_CITY_NAME_NIDAROS
9,6,LOC_CITY_NAME_KYOTO
11,7,LOC_CITY_NAME_TENOCHTITLAN
13,8,LOC_CITY_NAME_PARIS
15,9,LOC_CITY_NAME_GYEONGJU
17,10,LOC_CITY_NAME_HUNZA


In [33]:
merged = df.merge(leaders,
              how='left',
              on='City')
merged

,Game Turn,City,Production Added,Current Item,Current Production,Production Needed,Overflow,Player
0,2,LOC_CITY_NAME_ISTANBUL,5.0,UNIT_SCOUT,5.0,15,0.0,0
1,2,LOC_CITY_NAME_MIKISIW_WACIHK,6.4,BUILDING_MONUMENT,6.4,30,0.0,2
2,2,LOC_CITY_NAME_URUK,9.6,UNIT_SCOUT,9.6,15,0.0,3
3,2,LOC_CITY_NAME_OTTAWA,8.0,UNIT_SCOUT,8.0,15,0.0,4
4,2,LOC_CITY_NAME_NIDAROS,9.6,UNIT_SCOUT,9.6,15,0.0,5
...,...,...,...,...,...,...,...,...
802,25,LOC_CITY_NAME_YEREVAN,2.8,BUILDING_GRANARY,11.4,32,0.0,23
803,25,LOC_CITY_NAME_ANTANANARIVO,3.5,UNIT_TRADER,27.5,25,2.5,24
804,26,LOC_CITY_NAME_ISTANBUL,9.9,UNIT_SCOUT,22.6,15,7.6,0
805,26,LOC_CITY_NAME_BURSA,7.2,UNIT_BUILDER,21.6,27,0.0,0


In [35]:
merged[merged.duplicated(subset=['Game Turn', 'City', 'Player'], keep=False)]

,Game Turn,City,Production Added,Current Item,Current Production,Production Needed,Overflow,Player
361,14,LOC_CITY_NAME_TENOCHTITLAN,29.3,BUILDING_STONEHENGE,29.3,90,0.0,7
362,14,LOC_CITY_NAME_ATZCAPOTZALCO,11.2,DISTRICT_HOLY_SITE,24.4,34,0.0,7
363,14,LOC_CITY_NAME_TENOCHTITLAN,13.0,BUILDING_STONEHENGE,42.3,90,0.0,7
364,14,LOC_CITY_NAME_ATZCAPOTZALCO,13.0,DISTRICT_HOLY_SITE,37.4,34,3.4,7
388,15,LOC_CITY_NAME_MIKISIW_WACIHK,16.0,BUILDING_GRANARY,44.2,32,12.2,2
390,15,LOC_CITY_NAME_MIKISIW_WACIHK,26.2,UNIT_ARCHER,26.2,30,0.0,2
405,15,LOC_CITY_NAME_HUNZA,3.5,BUILDING_MONUMENT,31.5,30,1.5,10
406,15,LOC_CITY_NAME_HUNZA,15.5,BUILDING_GRANARY,15.5,32,0.0,10
410,15,LOC_CITY_NAME_SAMARKAND,15.0,UNIT_WARRIOR,30.0,20,0.0,14
411,15,LOC_CITY_NAME_SAMARKAND,14.0,BUILDING_GRANARY,14.0,32,0.0,14


In [20]:
df[df.duplicated(subset=['Game Turn', 'City', 'Production Added'], keep=False)]

,Game Turn,City,Production Added,Current Item,Current Production,Production Needed,Overflow


In [39]:
merged[merged['City'] == ' LOC_CITY_NAME_ANTIOCH']

,Game Turn,City,Production Added,Current Item,Current Production,Production Needed,Overflow,Player
20,2,LOC_CITY_NAME_ANTIOCH,6.0,UNIT_SLINGER,6.0,17,0.0,21
21,2,LOC_CITY_NAME_ANTIOCH,6.0,UNIT_SLINGER,6.0,17,0.0,7
36,3,LOC_CITY_NAME_ANTIOCH,6.0,UNIT_SLINGER,12.0,17,0.0,21
37,3,LOC_CITY_NAME_ANTIOCH,6.0,UNIT_SLINGER,12.0,17,0.0,7
60,4,LOC_CITY_NAME_ANTIOCH,6.0,UNIT_SLINGER,18.0,17,0.0,21
61,4,LOC_CITY_NAME_ANTIOCH,6.0,UNIT_SLINGER,18.0,17,0.0,7
87,5,LOC_CITY_NAME_ANTIOCH,2.0,BUILDING_MONUMENT,2.0,30,0.0,21
88,5,LOC_CITY_NAME_ANTIOCH,2.0,BUILDING_MONUMENT,2.0,30,0.0,7
115,6,LOC_CITY_NAME_ANTIOCH,2.0,BUILDING_MONUMENT,4.0,30,0.0,21
116,6,LOC_CITY_NAME_ANTIOCH,2.0,BUILDING_MONUMENT,4.0,30,0.0,7


In [38]:
df[df['City'] == ' LOC_CITY_NAME_ANTIOCH']

,Game Turn,City,Production Added,Current Item,Current Production,Production Needed,Overflow
20,2,LOC_CITY_NAME_ANTIOCH,6.0,UNIT_SLINGER,6.0,17,0.0
35,3,LOC_CITY_NAME_ANTIOCH,6.0,UNIT_SLINGER,12.0,17,0.0
58,4,LOC_CITY_NAME_ANTIOCH,6.0,UNIT_SLINGER,18.0,17,0.0
84,5,LOC_CITY_NAME_ANTIOCH,2.0,BUILDING_MONUMENT,2.0,30,0.0
111,6,LOC_CITY_NAME_ANTIOCH,2.0,BUILDING_MONUMENT,4.0,30,0.0
138,7,LOC_CITY_NAME_ANTIOCH,2.0,BUILDING_MONUMENT,6.0,30,0.0
169,8,LOC_CITY_NAME_ANTIOCH,2.0,BUILDING_MONUMENT,8.0,30,0.0
201,9,LOC_CITY_NAME_ANTIOCH,2.0,BUILDING_MONUMENT,10.0,30,0.0
234,10,LOC_CITY_NAME_ANTIOCH,2.0,BUILDING_MONUMENT,12.0,30,0.0
267,11,LOC_CITY_NAME_ANTIOCH,2.0,BUILDING_MONUMENT,14.0,30,0.0
